In [ ]:
import json
import pandas as pd
from numpy import array

import torch
import torch.nn
import torch.optim as optim

# Used in LTSMModel Class Instantiation
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir("/content/drive/MyDrive/Colab Notebooks/698/data/10step_1horizon")

In [ ]:
#%run data_prep_one_hero.ipynb

#torch.load(df_allhero, 'df_allhero.pt')
#torch.load(hero_ids, 'hero_ids.pt')
#torch.load(df_all_remain, 'df_all_remain.pt')

torch.load(df2_allhero, 'df2_allhero.pt')
torch.load(hero_ids2, 'hero_ids2.pt')
torch.load(df2_all_remain, 'df2_all_remain.pt')

      hero_id gold_t
2270       20   None
2271      120   None
2272       69   None
2273       73   None
2274       26   None
...       ...    ...
6915      136   None
6916      106   None
6917      137   None
6918       14   None
6919      112   None

[76 rows x 2 columns]
      hero_id gold_t
2270       20   None
2271      120   None
2272       69   None
2273       73   None
2274       26   None
...       ...    ...
6915      136   None
6916      106   None
6917      137   None
6918       14   None
6919      112   None

[76 rows x 2 columns]
Found 34 tensors with length 0 at indices: [810, 811, 812, 813, 814, 815, 816, 817, 818, 819, 1390, 1391, 1392, 1393, 1394, 1395, 1396, 1397, 1398, 1399, 2460, 2501, 2512, 2583, 5414, 5415, 5416, 5417, 5418, 5419, 5420, 5421, 5422, 5423]
Found 34 tensors with length 0 at indices: [810, 811, 812, 813, 814, 815, 816, 817, 818, 819, 1390, 1391, 1392, 1393, 1394, 1395, 1396, 1397, 1398, 1399, 2460, 2501, 2512, 2583, 5414, 5415, 5416, 5417, 5418, 5419

NameError: name 'X_scaled' is not defined

NameError: name 'X_scaled' is not defined

In [ ]:
#df_allhero[['gold_t']]

#df_allhero[['gold_t']] =

In [ ]:
df2_allhero_avglen

In [ ]:
len(df_allhero.iloc[0]['gold_t'])

### TimeSeriesDataset Class

In [ ]:
type(df_allhero[['gold_t']])

In [ ]:
import torch
from torch.utils.data import Dataset

class TimeSeriesDataset(Dataset):
    # Class to create our dataset
    def __init__(self, df, lookback):
        self.hero_ids = df['hero_id'].values # Declaring hero_id values
        self.time_series = df[['gold_t']]
        #[torch.tensor(ts) for ts in df['gold_t']] # Converting the time_series into Tensors
        self.max_length = max(len(ts) for ts in self.time_series) # Grabs max length of all the tensors to pad them with 0s later
        self.match_ids = df['match_id'] #Storing the match_id in case we want to view this later for more info
        self.lookback = lookback


    def __len__(self):
        return len(self.hero_ids) # Convenient length call


    def create_windows(self, timeseries):
        X, y = [], []
        for i in range(len(timeseries) - self.lookback):
            feature = timeseries[i:i+self.lookback]
            target = timeseries[i+1:i+self.lookback+1]
            X.append(feature)
            y.append(target)


        # print("Create Window X Obj:",X)
        # print("Create Window y Obj:",y)
        # print("Create Window Type X:", type(X))
        # print("Create Window Type y:", type(y))
        #print(len(X))
        #print(len(y))

        X = torch.tensor(X)
        y = torch.tensor(y)
        return X, y



    def __getitem__(self, idx):
        #print("1st Step __getitem__ State: ", idx)
        hero_id = self.hero_ids[idx]

        time_series = np.array(self.time_series.iloc[idx][0]).astype('float32') # Since the df_allhero 'gold_t' column is a column of lists, we take the
        #first row of the df_allhero with .iloc[idx]
        # then we access the the first element of the row, which is the list, with [0]
        # we convert it to a numpy array, and then convert values to float32
        # we do this to be compatible with the ConstantMinMaxScaler()

        match_id = self.match_ids[idx]



        scaled_time_series = ConstantMinMaxScaler(time_series, min_gold, max_gold)
        #print("Type of scaled_time_series:",type(scaled_time_series))
        length = len(scaled_time_series)

        X, y = self.create_windows(scaled_time_series)
        #print(X.shape, y.shape)
        # print("Post create_window:",idx)
        # print("Type of X", type(X))
        # print("Type of y", type(y))



        return hero_id, X, y



### Embedding Layer

Embedding module expects the input tensor to have a shape of (batch_size, sequence_length), where batch_size is the number of samples in a batch and sequence_length is the length of each input sequence

In [ ]:
import torch
import torch.nn as nn

class ProcessEmbedding(nn.Module):
    def __init__(self, df, embedding_dim, lookback):
        super(ProcessEmbedding, self).__init__()

        self.num_processes = len(df['hero_id'].unique()) # declaring number of different categories of time-series for dimensionialty reasons
        self.embedding_dim = embedding_dim # passing our embed size to be a class attribute
        self.process_embeddings = nn.Embedding(self.num_processes, embedding_dim)

        self.hero_id_to_idx = {hero_id: idx for idx, hero_id in enumerate(df['hero_id'].unique())}



    def forward(self, hero_id):
        process_ids = self.hero_id_to_idx[hero_id]
        process_ids = torch.tensor([process_ids])
        process_embeddings = self.process_embeddings(process_ids) #.unsqueeze(1).repeat(1, self.lookback, 1)

        # print("Process Embeddings shape:", process_embeddings.shape)
        # print("Process Embeddings tensor:", process_embeddings)

        return process_embeddings


### LSTM

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, process_embedding):
        super(LSTMModel, self).__init__() # ensures the correcty PyTorch class is also initialized

        self.hidden_size = hidden_size #hyper param
        self.num_layers = num_layers #hyper param

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True) # Actual LSTM creation
        self.fc = nn.Linear(hidden_size, output_size) # Linear Model creation
        self.process_embedding = process_embedding # Process Embedding


    def forward(self, batch):
        #print("LSTM Forward Method Batch Type: ",type(batch))
        # Since our Dataset class returns 3 objects, hero_id, X, y and the forward method only expects two, we have to
        #    tell our forward method to expect one object and unpack it
        hero_ids = batch[0][0] # the _ is a placeholder that doesn't save the last object in the batch which is the y tensor
        X = batch[1][0]
        X = X.unsqueeze(-1) # To match LSTM model's desired shape of (batch_size, seq_length, input_size)

        # print("LSTM Forward Method X Type: ",type(X))
        # print(" LSTM Forward Method - X Shape:", X.shape)
        # print("LSTM Forward Method Tensor X",X)
        # print("LSTM Forward Method hero_ids Type: ",type(hero_ids))
        # print("LSTM Forward Method hero_ids:", hero_ids)

        batch_size = X.size(0) # pulling dims from the tensor
        seq_length = X.size(1) # pulling dims from the tensor

        # print("LSTM Forward Method - batch_size", batch_size)

        # Get process embeddings for hero_ids
        process_embeddings = self.process_embedding(hero_ids)

        # print("LSTM Forward Method - Process Embeddings Shape Pre-Repeat:", process_embeddings.shape)
        # print("LSTM Forward Method - Process Embeddings:", process_embeddings)

        # Reshape process embeddings to match the input shape of LSTM
        # process_embeddings = process_embeddings.unsqueeze(1).repeat(1, seq_length, 1)
        process_embeddings = process_embeddings.repeat(batch_size, seq_length, 1) # changing process embedding shape to broadcast across the same number of samples in the X tensor
        # we do this to match the dimensions so that torch.cat will work
        # print("LSTM Forward Method - Process Embeddings Shape Post-Repeat:", process_embeddings.shape)
        # dim = -1, signifies concatenation across the last dimension (the feature dimension)
        combined_input = torch.cat((X,process_embeddings),dim=-1) #

        # print("Concat'd Time-Series + Embedding shape:", combined_input.shape)

        # Unsqueexing to ensure the time_series shape is 3D like our embedding processing is so that no issues are ran into with torch.cat below
        #time_series = time_series.unsqueeze(-1)

        #print("Time Series shape with extra dimension:", time_series.shape)

        # Concatenate process embeddings with time series data



        # Pack the padded sequences
        # Packing the padded Sequences is a way of optimizing computation times. We have padded the time series to all be the same length, even though some are only 20 or less
        # The packing indicates which are the real values in the time series so that the computation is only ran on those time steps. Details on how are unknown to me thus far.
        #packed_input = pack_padded_sequence(input_data, lengths, batch_first=True, enforce_sorted=False)


        # Initialize hidden state and cell state
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_size)


        #packed_output, _ = self.lstm(packed_input, (h0, c0))

        # Unpack the output
        #output, _ = pad_packed_sequence(packed_output, batch_first=True)

        # Pass combined input to LSTM layer
        output, _ = self.lstm(combined_input)

        # Take the last output of the LSTM
        out = self.fc(output[:, -1, :])

        return out

### Instantiating Classes and Parameters

In [ ]:
lookback = 10
#train_dataset = TimeSeriesDataset(df_allhero, lookback)
process_embedding = ProcessEmbedding(df2_allhero, embedding_dim=84, lookback=lookback) # we create the embedding vector on unsplit data to ensure all unique hero id's are contained

input_size = process_embedding.embedding_dim + 1 #84 + 1
hidden_size = 64
num_layers = 2
output_size = 1  # Assuming you want to predict a single value

model = LSTMModel(input_size, hidden_size, num_layers, output_size, process_embedding)

### Train Test Split

In [ ]:
# test_size = .30

# train_df, test_df = train_test_split(df_full, test_size=test_size, shuffle=False)



#### Dataset and Data Load

In [ ]:
batch_size = 1

train_dataset = TimeSeriesDataset(df2_allhero, lookback=lookback)
test_dataset = TimeSeriesDataset(df2_allhero_avglen, lookback=lookback)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))
# train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(train_dataset), batch_size=batch_size, shuffle=False)
# test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(test_dataset), batch_size=batch_size, shuffle=False)


In [ ]:
#len(df_allhero.iloc[32]['gold_t'])

In [ ]:
# # Iterate over the train_loader until the desired index
# for idx, (hero_ids, X, y) in enumerate(train_loader):
#     print(idx)
#     if idx == 32:
#         print('Gottem')
#         break

In [ ]:
# Training loop
num_epochs = 500


# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

best_test_loss = float('inf')
best_model_state = None

for epoch in range(num_epochs):
    model.train()

    train_loss = 0.0
    print(f"Epoch: {epoch}")

    for batch in train_loader: #for hero_ids, X, y in train_loader:

        hero_ids, X, y = batch #outputs = model(hero_ids, X)
        #print("Training Loop X Type", type(X))
        #print("Training Loop X shape", len(X))
        #print("Training Loop X", X)

        #print("Training Loop Heros Type", type(hero_ids))
        #print("Training Loop Heros shape", len(hero_ids))
        #print("Training Loop Heros", hero_ids)

        X = X[0]
        # print("Training Loop X Type", type(X))
        # print("Training Loop X shape", len(X))
        # print("Training Loop X", X)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(batch)

        y = y[0]
        # print("Training Loop Y Type", type(y))
        # print("Training Loop Y shape", len(y))
        # print("Training Loop Y", y)

        targets = y[:, -1]  # Assuming you want to predict the last value of each time series
        loss = criterion(outputs.squeeze(), targets.squeeze())

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * X.size(0)

    train_loss /= len(train_dataset)

    model.eval()
    test_loss = 0.0

    with torch.no_grad():
         for batch in test_loader: #for hero_ids, X, y in test_loader:
            # Forward pass
            hero_ids, X, y = batch
            X = X[0]
            y = y[0]
            outputs = model(batch) # outputs = model(hero_ids, X)
            targets = y[:, -1]  # Assuming you want to predict the last value of each time series
            loss = criterion(outputs.squeeze(), targets.squeeze())

            test_loss += loss.item() * X.size(0)

    test_loss /= len(test_dataset)

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}")

    if test_loss < best_test_loss:
      best_test_loss = test_loss
      best_model_state = model.state_dict()

torch.save(best_model_state, '10step_1horizon_best_model.pth')

In [ ]:
best_model_state = torch.load('10step_1horizon_best_model.pth')


In [ ]:
def create_windows(timeseries, lookback):
    X, y = [], []
    for i in range(len(timeseries) - lookback):
        feature = timeseries[i:i+lookback]
        target = timeseries[i+1:i+lookback+1]
        X.append(feature)
        y.append(target)

    X = torch.tensor(X)
    y = torch.tensor(y)
    return X, y

In [ ]:
# Assuming you have already trained the model and have the `model` object available
model.load_state_dict(best_model_state)
# Select a specific time series from df_all_remain
index = 8000  # Choose the index of the desired time series
selected_row = df_all_remain.iloc[index]

# Extract the necessary information from the selected row
hero_id = selected_row['hero_id']
time_series = np.array(selected_row['gold_t']).astype('float32')

# Scale the time series using the same scaling function used during training
scaled_time_series = ConstantMinMaxScaler(time_series, min_gold, max_gold)

# Create windows from the scaled time series
X, y = create_windows(scaled_time_series, lookback)

# Convert X and y to tensors
#X = torch.tensor(X)
#y = torch.tensor(y)

# Create a batch with the selected time series
batch = ((hero_id,), (X,), (y,))

# Set the model to evaluation mode
model.eval()

# Disable gradient computation
with torch.no_grad():
    # Forward pass
    outputs = model(batch)

    # Get the predicted values
    predicted = outputs.squeeze().numpy()


#Unscaling values
unscaled_targets = ConstantUnScaler(y[:,-1], min_gold, max_gold)
unscaled_preds = ConstantUnScaler(predicted, min_gold, max_gold)

# Print the predicted values
print("Targets:")
print(y[:,-1])
print("Predicted values:")
print(predicted)

# Print Unscaled Targets
print("Unscaled Targets")
print(unscaled_targets)
print("Unscaled Preds")
print(unscaled_preds)

In [ ]:
import matplotlib.pyplot as plt

preds_plot = np.ones_like(time_series) * np.nan
preds_plot[lookback:lookback+len(unscaled_preds)] = unscaled_preds
# plot
plt.plot(time_series, label = "Actual Values")
plt.plot(preds_plot, c='r', label = "Forecasting Predictions")
#plt.plot(test_plot, c='g', label = "Test Predictions")

plt.title(f"HeroID:{hero_id} Game Length: {len(time_series)} Minutes, Lookback Window: {lookback}")

# Add a label to the x-axis
plt.xlabel('Timestep (Minute)')

# Add a label to the y-axis
plt.ylabel('Gold')
plt.legend()
plt.show()

In [ ]:
import pandas as pd
from sklearn.metrics import mean_squared_error

# Create an empty dataframe to store the average RMSE scores for each hero
df_results_all = pd.DataFrame(columns=['hero_id', 'avg_rmse'])

# Get the unique hero_ids from the dataframe
hero_ids = df_all_remain['hero_id'].unique()

# Loop over each hero_id
for hero_id in hero_ids:
    # Filter the dataframe for the current hero_id
    hero_df = df_all_remain[df_all_remain['hero_id'] == hero_id]

    # Create empty lists to store the unscaled targets and predictions for the current hero
    unscaled_targets_list = []
    unscaled_preds_list = []

    # Loop over each record for the current hero
    for index, row in hero_df.iterrows():
        # Extract the necessary information from the row
        time_series = np.array(row['gold_t']).astype('float32')

        # Scale the time series using the same scaling function used during training
        scaled_time_series = ConstantMinMaxScaler(time_series, min_gold, max_gold)

        # Create windows from the scaled time series
        X, y = create_windows(scaled_time_series, lookback)

        # Create a batch with the selected time series
        batch = ((hero_id,), (X,), (y,))

        # Set the model to evaluation mode
        model.eval()

        # Disable gradient computation
        with torch.no_grad():
            # Forward pass
            outputs = model(batch)

            # Get the predicted values
            predicted = outputs.squeeze().numpy()

        # Unscaling values
        unscaled_target = ConstantUnScaler(y[-1], min_gold, max_gold)
        unscaled_pred = ConstantUnScaler(predicted[-1], min_gold, max_gold)

        # Append the last unscaled target and prediction to the respective lists
        unscaled_targets_list.append(unscaled_target)
        unscaled_preds_list.append(unscaled_pred)

    # Calculate the RMSE for the current hero
    rmse = np.sqrt(mean_squared_error(unscaled_targets_list, unscaled_preds_list))

    # Create a dataframe for the current hero with RMSE value
    df_results_hero = pd.DataFrame({'rmse': [rmse]})

    # Save the dataframe with the hero_id in the name
    df_results_hero.to_csv(f'df_results_{hero_id}.csv', index=False)

    # Append the hero_id and RMSE to the df_results_all dataframe
    df_results_all = df_results_all.append({'hero_id': hero_id, 'avg_rmse': rmse}, ignore_index=True)

# Save the df_results_all dataframe
df_results_all.to_csv('df_results_all.csv', index=False)

In [ ]:
#test = pd.DataFrame(df_allhero[['gold_t']].iloc[0])
#test